In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("transfusion_10%.csv")

In [3]:
df

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,NaN,98.0,1.0
1,0.0,13.0,3250.0,28.0,1.0
2,1.0,16.0,4000.0,NaN,1.0
3,2.0,20.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,23.0,2.0,500.0,NaN,0.0
744,NaN,2.0,NaN,52.0,0.0
745,NaN,NaN,NaN,62.0,0.0
746,39.0,1.0,250.0,NaN,0.0


In [4]:
df.dtypes

Recency (months)                              float64
Frequency (times)                             float64
Monetary (c.c. blood)                         float64
Time (months)                                 float64
whether he/she donated blood in March 2007    float64
dtype: object

In [5]:
df.isnull().sum()

Recency (months)                              107
Frequency (times)                              78
Monetary (c.c. blood)                          94
Time (months)                                  95
whether he/she donated blood in March 2007      0
dtype: int64

In [6]:
df.isnull().sum().sum()

374

In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Recency (months)                              14.304813
Frequency (times)                             10.427807
Monetary (c.c. blood)                         12.566845
Time (months)                                 12.700535
whether he/she donated blood in March 2007     0.000000
dtype: float64

In [8]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 3740
nilai hilang : 374
persentase : 10.0


In [9]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [10]:
from collections import Counter

def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count <= 2 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None    

def Final(df,col,k):
    terdekat = []
    mean = 0
    minimum = None
    maximum = None
    fuzzy_val = []
    euc_val = []
    
    for val in range(k):
        terdekat.append(euc_dist[col][val])
    
    for x,y in terdekat:
        if (minimum is None or maximum is None):
            minimum = y
            maximum = y
        elif(y<minimum):
            minimum = y
        elif(y>maximum):
            maximum = y
        mean += y
        euc_val.append(y)
    mean /= len(terdekat)
    euc_val.sort()
    print(euc_val)
    
    fuzzy = hitung(euc_val,3)
    print("min : ",minimum)
    print("max : ",maximum)
    print("mean : ",mean)
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_val.append((y,fuzzy_triangular(y,minimum,mean,maximum)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)

    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
    
    dummy.iloc[index,col] = tertimbang
    print("row, col :", index,col)
    print(" ")

In [11]:
def fuzzy_triangular(val, minimum, mean, maximum):
   if val <= minimum or val >= maximum: 
        return 0.0
   elif minimum < val <= mean:
        return float((val - minimum)/(mean - minimum)) 
   elif mean < val < maximum:
        return float((maximum - val)/(maximum - mean))

In [12]:
data = df.iloc[:, :-1]
df2 = df.copy()
dummy = df2.fillna(0)
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,0.0,98.0,1.0
1,0.0,13.0,3250.0,28.0,1.0
2,1.0,16.0,4000.0,0.0,1.0
3,2.0,20.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,23.0,2.0,500.0,0.0,0.0
744,0.0,2.0,0.0,52.0,0.0
745,0.0,0.0,0.0,62.0,0.0
746,39.0,1.0,250.0,0.0,0.0


In [13]:
euc_dist = {}

In [14]:
for i in range(df2.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,3)
dummy

Kolom : 1
[1.0, 2.0, 4.0]
min :  1.0
max :  4.0
mean :  2.3333333333333335
fuzzy :  [(2.0, 0.7499999999999999), (4.0, 0.0), (1.0, 0.0)]
fuzzy :  None
pembilang :  1.4999999999999998
penyebut :  0.7499999999999999
rata-rata tertimbang :  2.0
row, col : 1 0
 
[0.0, 4.0, 4.0]
median :  2
min :  0.0
max :  4.0
mean :  2.6666666666666665
fuzzy :  [(4.0, 4.0), (0.0, 4.0), (4.0, 4.0)]
fuzzy :  4.0
pembilang :  32.0
penyebut :  12.0
rata-rata tertimbang :  4.0
row, col : 8 0
 
[1.0, 5.0, 11.0]
min :  1.0
max :  11.0
mean :  5.666666666666667
fuzzy :  [(11.0, 0.0), (1.0, 0.0), (5.0, 0.8571428571428571)]
fuzzy :  None
pembilang :  4.285714285714286
penyebut :  0.8571428571428571
rata-rata tertimbang :  5.0
row, col : 10 0
 
[0.0, 2.0, 2.0]
median :  2
min :  0.0
max :  2.0
mean :  1.3333333333333333
fuzzy :  [(2.0, 2.0), (2.0, 2.0), (0.0, 2.0)]
fuzzy :  2.0
pembilang :  8.0
penyebut :  6.0
rata-rata tertimbang :  2.0
row, col : 11 0
 
[0.0, 4.0, 7.0]
min :  0.0
max :  7.0
mean :  3.6666666666666

[0.0, 11.0, 23.0]
min :  0.0
max :  23.0
mean :  11.333333333333334
fuzzy :  [(0.0, 0.0), (11.0, 0.9705882352941176), (23.0, 0.0)]
fuzzy :  None
pembilang :  10.676470588235293
penyebut :  0.9705882352941176
rata-rata tertimbang :  11.0
row, col : 258 0
 
[0.0, 11.0, 16.0]
min :  0.0
max :  16.0
mean :  9.0
fuzzy :  [(0.0, 0.0), (11.0, 0.7142857142857143), (16.0, 0.0)]
fuzzy :  None
pembilang :  7.857142857142858
penyebut :  0.7142857142857143
rata-rata tertimbang :  11.0
row, col : 259 0
 
[0.0, 14.0, 14.0]
median :  2
min :  0.0
max :  14.0
mean :  9.333333333333334
fuzzy :  [(0.0, 14.0), (14.0, 14.0), (14.0, 14.0)]
fuzzy :  14.0
pembilang :  392.0
penyebut :  42.0
rata-rata tertimbang :  14.0
row, col : 271 0
 
[0.0, 2.0, 18.0]
min :  0.0
max :  18.0
mean :  6.666666666666667
fuzzy :  [(0.0, 0.0), (2.0, 0.3), (18.0, 0.0)]
fuzzy :  None
pembilang :  0.6
penyebut :  0.3
rata-rata tertimbang :  2.0
row, col : 286 0
 
[0.0, 9.0, 11.0]
min :  0.0
max :  11.0
mean :  6.666666666666667
fuz

[0.0, 2.0, 4.0]
min :  0.0
max :  4.0
mean :  2.0
fuzzy :  [(0.0, 0.0), (4.0, 0.0), (2.0, 1.0)]
fuzzy :  None
pembilang :  2.0
penyebut :  1.0
rata-rata tertimbang :  2.0
row, col : 515 0
 
[0.0, 2.0, 9.0]
min :  0.0
max :  9.0
mean :  3.6666666666666665
fuzzy :  [(0.0, 0.0), (2.0, 0.5454545454545455), (9.0, 0.0)]
fuzzy :  None
pembilang :  1.090909090909091
penyebut :  0.5454545454545455
rata-rata tertimbang :  2.0
row, col : 516 0
 
[0.0, 3.0, 4.0]
min :  0.0
max :  4.0
mean :  2.3333333333333335
fuzzy :  [(0.0, 0.0), (4.0, 0.0), (3.0, 0.6000000000000001)]
fuzzy :  None
pembilang :  1.8000000000000003
penyebut :  0.6000000000000001
rata-rata tertimbang :  3.0
row, col : 519 0
 
[0.0, 3.0, 7.0]
min :  0.0
max :  7.0
mean :  3.3333333333333335
fuzzy :  [(0.0, 0.0), (7.0, 0.0), (3.0, 0.8999999999999999)]
fuzzy :  None
pembilang :  2.6999999999999997
penyebut :  0.8999999999999999
rata-rata tertimbang :  3.0
row, col : 523 0
 
[0.0, 2.0, 2.0]
median :  2
min :  0.0
max :  2.0
mean :  1.3

[10.0, 10.0, 10.0]
median :  2
min :  10.0
max :  10.0
mean :  10.0
fuzzy :  [(10.0, 10.0), (10.0, 10.0), (10.0, 10.0)]
fuzzy :  10.0
pembilang :  300.0
penyebut :  30.0
rata-rata tertimbang :  10.0
row, col : 12 1
 
[11.0, 11.0, 11.0]
median :  2
min :  11.0
max :  11.0
mean :  11.0
fuzzy :  [(11.0, 11.0), (11.0, 11.0), (11.0, 11.0)]
fuzzy :  11.0
pembilang :  363.0
penyebut :  33.0
rata-rata tertimbang :  11.0
row, col : 21 1
 
[9.0, 9.0, 9.0]
median :  2
min :  9.0
max :  9.0
mean :  9.0
fuzzy :  [(9.0, 9.0), (9.0, 9.0), (9.0, 9.0)]
fuzzy :  9.0
pembilang :  243.0
penyebut :  27.0
rata-rata tertimbang :  9.0
row, col : 46 1
 
[7.0, 7.0, 7.0]
median :  2
min :  7.0
max :  7.0
mean :  7.0
fuzzy :  [(7.0, 7.0), (7.0, 7.0), (7.0, 7.0)]
fuzzy :  7.0
pembilang :  147.0
penyebut :  21.0
rata-rata tertimbang :  7.0
row, col : 57 1
 
[6.0, 6.0, 6.0]
median :  2
min :  6.0
max :  6.0
mean :  6.0
fuzzy :  [(6.0, 6.0), (6.0, 6.0), (6.0, 6.0)]
fuzzy :  6.0
pembilang :  108.0
penyebut :  18.0
rat

[0.0, 0.0, 3.0]
median :  2
min :  0.0
max :  3.0
mean :  1.0
fuzzy :  [(0.0, 0.0), (3.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 371 1
 
[0.0, 4.0, 4.0]
median :  2
min :  0.0
max :  4.0
mean :  2.6666666666666665
fuzzy :  [(0.0, 4.0), (4.0, 4.0), (4.0, 4.0)]
fuzzy :  4.0
pembilang :  32.0
penyebut :  12.0
rata-rata tertimbang :  4.0
row, col : 373 1
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 382 1
 
[0.0, 0.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.3333333333333333
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (1.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 383 1
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut : 

...

[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
row, col : 590 4
 
[0.0, 0.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.3333333333333333
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (1.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 591 4
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
row, col : 592 4
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
row, col : 594 4
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0


[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
row, col : 635 4
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
row, col : 636 4
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
row, col : 637 4
 
[0.0, 0.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.3333333333333333
fuzzy :  [(0.0, 0.0), (1.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 638 4
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0


[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
row, col : 679 4
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
row, col : 681 4
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
row, col : 682 4
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 683 4
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.

[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
row, col : 722 4
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
row, col : 723 4
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 724 4
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 725 4
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,11000.0,98.0,1.0
1,2.0,13.0,3250.0,28.0,1.0
2,1.0,16.0,4000.0,64.0,1.0
3,2.0,20.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,1.0
...,...,...,...,...,...
743,23.0,2.0,500.0,23.0,1.0
744,4.0,2.0,500.0,52.0,1.0
745,4.0,2.0,500.0,62.0,1.0
746,39.0,1.0,250.0,38.0,1.0


In [15]:
dummy.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
count,748.000000,748.000000,748.000000,748.000000,748.0
mean,9.118984,5.501337,1366.644385,34.153743,1.0
std,7.892722,5.843451,1424.710345,24.345772,0.0
min,1.000000,1.000000,250.000000,2.000000,1.0
25%,2.000000,2.000000,500.000000,16.000000,1.0
50%,5.000000,4.000000,1000.000000,28.000000,1.0
75%,14.000000,7.000000,1750.000000,49.000000,1.0
max,74.000000,50.000000,11500.000000,98.000000,1.0


# Klasifikasi

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [17]:
x = dummy.values
y = df.iloc[:, -1].values

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=94)

In [19]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [20]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[132  18]
 [ 23  14]]
0.7807486631016043
